# ***Advanced tools for large datasets***
#### Author: Gabriel Luan Souza de Oliveira

#### Last update: 15/02/2022 23:22:00

## ***For local installation***:
A useful commmand to avoid any import problems (like circular imports) with holoviews, bokh and datashader, use this command on your terminal:

`pip install holoviews[all]`

## ***For jupyterhub installation***:
Every time systems crash and your spawn is erased, you have to re-upgrade packages. To do this, just type the commmands below in the terminal:

`pip install --upgrade holoviews[all]`

## ***True Documentation***:
https://holoviews.org/Reference_Manual/index.html

In [ ]:
import numpy as np
import healpy as hp
from astropy.table import Table
import bokeh
from bokeh.io import output_notebook, show
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import datashade, shade, dynspread, spread, rasterize
import inspect
hv.extension('bokeh')
output_notebook()
from gavodb import DBManager

## ***Sky and pixels initial information***:

In [ ]:
nside=4096
steradian_in_degrees = (180./np.pi)**2
sky_area = 4*np.pi*steradian_in_degrees   #Area of an sphere measured in degrees^2. Read more on https://www.mathsisfun.com/geometry/steradian.html
npixels = hp.nside2npix(nside)            #Númeto de pixeis em um Healpix pixelization scheme => 12.0*(nside)**2.0
area_of_each_pixel = sky_area/(npixels)   #In degrees^2.
pix_area_arcmin = 3600*area_of_each_pixel #Pixel area in (minutes of degree)².

# ***Loading catalog***

### Calling DBManager

In [ ]:
dbm = DBManager()

In [ ]:
%%time
catalog = dbm.table_loading('6838',1,['hpix_4096', 'ra', 'dec', 'z_best', 'err_z'],False,False)
#catalog.write('catalog', format='csv')

In [ ]:
def loading_catalog(catalog, seed, p_0):
    np.random.seed(seed)
    random_indexs = np.random.choice(2, size=len(catalog), p=[p_0,1-p_0]).astype('bool')
    catalog_reduced = catalog[random_indexs]
    
    del catalog
    
    return catalog_reduced

seed = 1024
p_0 = 0.99

#catalog_complete = loading_catalog(catalog, seed, 0)
catalog_reduced  = loading_catalog(catalog, seed, 0.99)
#print(f'Length of catalog_complete: {len(catalog_complete)} rows.')
print(f'Length of catalog_reduced: {len(catalog_reduced)} rows.')

# Object density map

## Option dicts

In [ ]:
#hv.help(hv.Points)
point_options = {'title': 'Object count map',
                'toolbar':'right',
                'color':'z_best',
                'cmap': 'hot',
                'marker': 'o',
                'size':0.5,
                'xlabel':'R.A.(degrees)',
                'ylabel': 'DEC. (degrees)',
                'bgcolor': '#f7f7f7',
                'colorbar': True,
                'clabel': 'N° of objects per pixel',
                'logz': False,
                }

rasterize_options = {'title': 'Object density map',
                    'toolbar':'right',
                    'xlabel':'R.A.(degrees)',
                    'ylabel': 'DEC. (degrees)',
                    'bgcolor': '#f7f7f7',
                    'colorbar': True,
                   }

datashade_options = {'title': 'Object density map',
                    'toolbar':'right',
                    'xlabel':'R.A.(degrees)',
                    'ylabel': 'DEC. (degrees)',
                    'bgcolor': '#f7f7f7',
                    
                   }
#hv.help(rasterize)

## Count

In [ ]:
def count(catalog):
    count_table = Table()
    count_table['pixels'] = np.unique(catalog['hpix_4096'].astype('int'))

    count = np.bincount(catalog['hpix_4096'].astype('int'))
    n_obj = np.array([count[each] for each in count_table['pixels']])


    ra,dec = hp.pix2ang(nside=4096, ipix=count_table['pixels'], lonlat = True, nest = True)
    ra[ra>180] -= 360

    count_table['ra'] = ra
    count_table['dec'] = dec
    count_table['n_obj'] = n_obj
    
    return count_table

#print(count(catalog_reduced))

## Point plot

In [ ]:
hv.output(size=150)
catalog_reduced = loading_catalog(catalog, seed, 0.9)
#count_table = count(catalog_reduced)
#data_points = zip(count_table['ra'], count_table['dec'], count_table['n_obj'])
#points = hv.Points(data_points, vdims = ['N° of objects per pixel'])

print('Observações sobre hv.Points:\n\n'\
       '- Sem contagem de objetos manual não é possível fazer o mapa de densidades, apenas plota pontinhos\n'\
       '- Pontos mantém o mesmo tamanho (que pode ser escolhido), independentemente do zoom aplicado.\n'\
       '- Permite fazer um histograma de praxe.\n'\
       '- Não consegue rodar para grande quantidade de pontos a menos que seu computador tenha 128 Gb de RAM\n'\
       '  e você use apenas uma aba aberta no Chrome \U0001F605.')
#points.opts(**point_options).hist(bins = 'fd')
data_points = zip(catalog_reduced['ra'], catalog_reduced['dec'],catalog_reduced['z_best'])
points = hv.Points(data_points, vdims = ['z_best'])
points.opts()

## Rasterize plot

In [ ]:
hv.output(size=150)
catalog_reduced = loading_catalog(catalog, seed, 0)
data_points = zip(catalog_reduced['ra'], catalog_reduced['dec'])
points = hv.Points(data_points)
rasterize(points).opts(cmap="hot_r", cnorm="eq_hist").relabel().opts(**rasterize_options).hist()

## Datashade plot

In [ ]:
hv.output(size=150)
catalog_reduced = loading_catalog(catalog, seed, 0)
data_points = zip(catalog_reduced['ra'], catalog_reduced['dec'])
points = hv.Points(data_points)
datashade(points).opts(**datashade_options)

## Spread